In [1]:
from astroquery.mast import Observations
import numpy as np

import pandas as pd
from astropy.table import Table, vstack, hstack

import shapely


from datetime import datetime as dt
from bokeh.io import output_notebook, show, vplot
from bokeh.plotting import figure
from bokeh.layouts import column

# First, get the grism data and it's products for Prop 13871:

In [2]:
obsTable = Observations.query_criteria(filters=["G141", "G102"],dataRights=['PUBLIC'], proposal_id=['13871'] )
obsTableProducts = Observations.get_product_list(obsTable)

print obsTable
#obsTable_pandas = obsTable.to_pandas()
#print obsTable_pandas['filters'], obsTable_pandas['obsid']
print obsTableProducts

dataproduct_type calib_level obs_collection ...   obsid      objID   
---------------- ----------- -------------- ... ---------- ----------
        spectrum           3            HST ... 2003858743 2012924107
        spectrum           3            HST ... 2003858745 2012924109
        spectrum           3            HST ... 2003858747 2012924111
        spectrum           3            HST ... 2004305738 2014141761
        spectrum           3            HST ... 2004305740 2014141763
        spectrum           3            HST ... 2004305742 2014141765
  obsID    obs_collection dataproduct_type ...     productFilename        size  
---------- -------------- ---------------- ... ----------------------- ---------
2002998239            HST         spectrum ... icka04pcq_flt_hlet.fits      8640
2002998239            HST         spectrum ...      icka04pcq_ima.fits 157743360
2002998239            HST         spectrum ...      icka04pcq_spt.fits   1152000
2002998239            HST         s

# Filter out the FLT files:

In [3]:
flt_filter = Observations.filter_products(obsTableProducts,mrp_only=False,productSubGroupDescription=["FLT"])
print flt_filter.to_pandas()

         obsID obs_collection dataproduct_type     obs_id  \
0   2002998239            HST         spectrum  ICKA04PCQ   
1   2002998240            HST         spectrum  ICKA04PDQ   
2   2002998243            HST         spectrum  ICKA04PLQ   
3   2002998244            HST         spectrum  ICKA04PNQ   
4   2002998247            HST         spectrum  ICKA05PJQ   
5   2002998248            HST         spectrum  ICKA05PKQ   
6   2002998251            HST         spectrum  ICKA05PSQ   
7   2002998252            HST         spectrum  ICKA05PUQ   
8   2002998255            HST         spectrum  ICKA06G7Q   
9   2002998256            HST         spectrum  ICKA06G8Q   
10  2002998259            HST         spectrum  ICKA06GGQ   
11  2002998260            HST         spectrum  ICKA06GIQ   
12  2004300702            HST         spectrum  ICKA01T7Q   
13  2004300703            HST         spectrum  ICKA01T8Q   
14  2004300706            HST         spectrum  ICKA01TGQ   
15  2004300707          

# Combine the two table outputs - it does this by the obsID:

In [4]:
all_prod_13871_grism_flts = (hstack([obsTable, flt_filter],join_type='inner'))

print all_prod_13871_grism_flts.columns
all_prod_13871_grism_flts_pandas = all_prod_13871_grism_flts.to_pandas()
print all_prod_13871_grism_flts_pandas

<TableColumns names=('dataproduct_type_1','calib_level','obs_collection_1','obs_id_1','target_name','s_ra','s_dec','t_min','t_max','t_exptime','wavelength_region','filters','em_min','em_max','target_classification','obs_title','t_obs_release','dataRights','mtFlag','srcDen','instrument_name','proposal_pi','proposal_id','project_1','s_region','jpegURL','dataURL','obsid','objID','obsID','obs_collection_2','dataproduct_type_2','obs_id_2','description','type','dataURI','productType','productGroupDescription','productSubGroupDescription','productDocumentationURL','project_2','prvversion','productFilename','size')>
  dataproduct_type_1  calib_level obs_collection_1   obs_id_1 target_name  \
0           spectrum            3              HST  ICKA04030    GN-Z10-1   
1           spectrum            3              HST  ICKA05030    GN-Z10-1   
2           spectrum            3              HST  ICKA06030    GN-Z10-1   
3           spectrum            3              HST  ICKA01030    GN-Z10-1   

# Now we want to get the visit numbers into the table:

In [5]:
visit_number = []
for curr_file in xrange(len(all_prod_13871_grism_flts_pandas['productFilename'])):
    visit_number.append(all_prod_13871_grism_flts_pandas['productFilename'][curr_file][4:6])

all_prod_13871_grism_flts_pandas['visit_number'] = visit_number

print all_prod_13871_grism_flts_pandas

  dataproduct_type_1  calib_level obs_collection_1   obs_id_1 target_name  \
0           spectrum            3              HST  ICKA04030    GN-Z10-1   
1           spectrum            3              HST  ICKA05030    GN-Z10-1   
2           spectrum            3              HST  ICKA06030    GN-Z10-1   
3           spectrum            3              HST  ICKA01030    GN-Z10-1   
4           spectrum            3              HST  ICKA02030    GN-Z10-1   
5           spectrum            3              HST  ICKA03030    GN-Z10-1   

         s_ra      s_dec        t_min        t_max    t_exptime      ...       \
0  189.106083  62.242056  57064.45186  57064.54036  5311.745971      ...        
1  189.106083  62.242056  57068.83606  57068.92959  5311.745971      ...        
2  189.106083  62.242056  57066.91493  57067.01162  5311.745971      ...        
3  189.106083  62.242056  57107.28618  57107.43866  5311.745971      ...        
4  189.106083  62.242056  57114.36929  57114.45788  531

# Split the table by visit number: 

In [6]:
grouped_visits = all_prod_13871_grism_flts_pandas.groupby(['visit_number'])
grouped_13871_grism_flts = [grouped_visits.get_group(x) for x in grouped_visits.groups]
print grouped_13871_grism_flts

[  dataproduct_type_1  calib_level obs_collection_1   obs_id_1 target_name  \
0           spectrum            3              HST  ICKA04030    GN-Z10-1   
1           spectrum            3              HST  ICKA05030    GN-Z10-1   
2           spectrum            3              HST  ICKA06030    GN-Z10-1   
3           spectrum            3              HST  ICKA01030    GN-Z10-1   

         s_ra      s_dec        t_min        t_max    t_exptime      ...       \
0  189.106083  62.242056  57064.45186  57064.54036  5311.745971      ...        
1  189.106083  62.242056  57068.83606  57068.92959  5311.745971      ...        
2  189.106083  62.242056  57066.91493  57067.01162  5311.745971      ...        
3  189.106083  62.242056  57107.28618  57107.43866  5311.745971      ...        

                                         dataURI productType  \
0  mast:HST/product/icka04pcq/icka04pcq_flt.fits     SCIENCE   
1  mast:HST/product/icka04pdq/icka04pdq_flt.fits     SCIENCE   
2  mast:HST/pro

# S_region overlaps? 

I haven't figured out how to query what images overlay each other using the s_region yet (or if that is even possible) - but I figure if we do a query around the RA and DEC I can later use the s_regions to create overlays and see which overlap and how. 

Here I do a query with two degrees around the RA and DEC of this visit: 

In [7]:
visit_test_one = grouped_13871_grism_flts[0]
print visit_test_one['s_dec'], visit_test_one['s_ra']
dec_one = visit_test_one['s_dec'][0]
ra_one = visit_test_one['s_ra'][0]

min_dec = dec_one - 1.0
max_dec = dec_one + 1.0

min_ra = ra_one - 1.0
max_ra = ra_one + 1.0

0    62.242056
1    62.242056
2    62.242056
3    62.242056
Name: s_dec, dtype: float64 0    189.106083
1    189.106083
2    189.106083
3    189.106083
Name: s_ra, dtype: float64


In [8]:
obsTableDecRA = Observations.query_criteria(filters=["G141", "G102"],dataRights=['PUBLIC'], obs_collection = ['HST'],
                                            s_dec =[min_dec, max_dec], s_ra=[min_ra, max_ra],instrument_name=['WFC3/IR'])

In [9]:
print obsTableDecRA.to_pandas()
proposals = list(np.unique(obsTableDecRA['proposal_id']))
print proposals

    dataproduct_type  calib_level obs_collection     obs_id target_name  \
0           spectrum            3            HST  IB3701060   GNGRISM11   
1           spectrum            3            HST  IB3702060   GNGRISM12   
2           spectrum            3            HST  IB3703060   GNGRISM13   
3           spectrum            3            HST  IB3704060   GNGRISM14   
4           spectrum            3            HST  IB3705060   GNGRISM21   
5           spectrum            3            HST  IB3706060   GNGRISM22   
6           spectrum            3            HST  IB3707060   GNGRISM23   
7           spectrum            3            HST  IB3708060   GNGRISM24   
8           spectrum            3            HST  IB3709060   GNGRISM31   
9           spectrum            3            HST  IB3710060   GNGRISM32   
10          spectrum            3            HST  IB3711060   GNGRISM33   
11          spectrum            3            HST  IB3712060   GNGRISM34   
12          spectrum     

# Direct images that overlap with our visit:

In [10]:
obsTableDecRA_directimages = Observations.query_criteria(dataproduct_type=['image'],dataRights=['PUBLIC'], obs_collection = ['HST'],
                                            s_dec =[min_dec, max_dec], s_ra=[min_ra, max_ra])
#obsTableProductsDecRA_directimages = Observations.get_product_list(obsTableDecRA_directimages)

In [11]:
#all_prod_directimage = (hstack([obsTableDecRA_directimages, obsTableProductsDecRA_directimagesDecRA],join_type='inner'))
#all_DecRA_directimages_pandas = all_prod_directimage.to_pandas()
obsTableDecRA_directimages_pandas = obsTableDecRA_directimages.to_pandas()

In [12]:
print np.unique(obsTableDecRA_directimages_pandas['filters'])
print np.unique(obsTableDecRA_directimages_pandas['proposal_id'])
print(obsTableDecRA_directimages_pandas.loc[obsTableDecRA_directimages_pandas['proposal_id'] == '13871'])

['F098M' 'F105W' 'F108N' 'F110W' 'F113N' 'F125W' 'F127M' 'F139M' 'F140W'
 'F150LP;F150LP' 'F153M' 'F160W' 'F164N' 'F166N' 'F187N' 'F190N' 'F196N'
 'F200LP' 'F200N' 'F205W' 'F212N' 'F215N' 'F220W' 'F222M' 'F237M' 'F240M'
 'F275W' 'F300W' 'F336W' 'F350LP' 'F435W' 'F438W' 'F450W' 'F475W' 'F547M'
 'F555W' 'F555W;F606W' 'F606W' 'F625W' 'F657N' 'F658N' 'F675W' 'F702W'
 'F763M' 'F775W' 'F814W' 'F814W;F791W' 'F845M' 'F850LP' 'F892N' 'MIRFUV'
 'MIRNUV' 'MIRVIS']
['10000' '10085' '10096' '10189' '10339' '10491' '10530' '10872' '11082'
 '11143' '11144' '11191' '11192' '11594' '11600' '12166' '12442' '12443'
 '12444' '12445' '12461' '12479' '12960' '13063' '13364' '13420' '13773'
 '13779' '13871' '13872' '14227' '14258' '4804' '6254' '6313' '6337' '6340'
 '6363' '6473' '6802' '7048' '7203' '7235' '7410' '7452' '7588' '7675'
 '7676' '7701' '7714' '7781' '7782' '7783' '7786' '7807' '7815' '7817'
 '7883' '7900' '7907' '7908' '7909' '7910' '7918' '7919' '7920' '7921'
 '7922' '8013' '8057' '8062' '8064

It takes too long to download all the products for all the direct images that match the above, it's also obvious from the various dataURL which of these rows match with our Visit 04 from our original searches - but it's not using the same naming convention that the flt files do. 

Another way to check is to use the t_min and t_max of the direct images in comparison with the grism images: 

In [13]:
print "t_min of grism data: "
print visit_test_one['t_min']

print "t_max of grism data: "
print visit_test_one['t_max']

t_min of grism data: 
0    57064.45186
1    57068.83606
2    57066.91493
3    57107.28618
Name: t_min, dtype: float64
t_max of grism data: 
0    57064.54036
1    57068.92959
2    57067.01162
3    57107.43866
Name: t_max, dtype: float64


In [14]:
direct_images_of_our_prop = obsTableDecRA_directimages_pandas.loc[obsTableDecRA_directimages_pandas['proposal_id'] == '13871']
print "t_min of direct data: "
print direct_images_of_our_prop['t_min']

print "t_max of direct data: "
print direct_images_of_our_prop['t_max']

t_min of direct data: 
6229    57064.44911
6230    57068.83331
6231    57066.91218
7298    57064.44985
7299    57068.83405
7300    57066.91292
8095    57107.28343
8096    57114.36654
8097    57115.84677
8101    57107.28417
8102    57114.36728
8103    57115.84751
Name: t_min, dtype: float64
t_max of direct data: 
6229    57064.54252
6230    57068.93176
6231    57067.01378
7298    57064.54122
7299    57068.93045
7300    57067.01248
8095    57107.44082
8096    57114.46005
8097    57115.93465
8101    57107.43953
8102    57114.45874
8103    57115.93336
Name: t_max, dtype: float64


In [15]:
print visit_test_one['dataURL']
print direct_images_of_our_prop['dataURL']

0    mast:HST/exposure/icka04030
1    mast:HST/exposure/icka05030
2    mast:HST/exposure/icka06030
3    mast:HST/exposure/icka01030
Name: dataURL, dtype: object
6229    mast:HST/exposure/icka04020
6230    mast:HST/exposure/icka05020
6231    mast:HST/exposure/icka06020
7298    mast:HST/exposure/jcka04010
7299    mast:HST/exposure/jcka05010
7300    mast:HST/exposure/jcka06010
8095    mast:HST/exposure/icka01020
8096    mast:HST/exposure/icka02020
8097    mast:HST/exposure/icka03020
8101    mast:HST/exposure/jcka01010
8102    mast:HST/exposure/jcka02010
8103    mast:HST/exposure/jcka03010
Name: dataURL, dtype: object


In [16]:
direct_image_tmin = direct_images_of_our_prop['t_min']
place_list = [1] * len(direct_image_tmin)
grism_tmin = visit_test_one['t_min']
place2 = [1] * len(grism_tmin)

In [17]:
p = figure(plot_width=1000, plot_height=600)

p.xaxis.axis_label="Exposure Start"

p.scatter(direct_image_tmin, place_list, size=5, line_color="navy", fill_color="teal", fill_alpha=0.5)
p.scatter(grism_tmin,place2, size=5, line_color="navy", fill_color="orange", fill_alpha=0.7)

show(p)

In [18]:
obsTableDecRA_pandas = obsTableDecRA.to_pandas()

In [19]:
s_regions_DecRA = obsTableDecRA_pandas['s_region']
print s_regions_DecRA

0      POLYGON -170.96938183999998 62.20174028 -171.0...
1      POLYGON -170.92170848 62.22415487 -170.9936149...
2      POLYGON -170.87392052 62.24640496 -170.9458802...
3      POLYGON -170.82683977 62.27000459 -170.9011183...
4      POLYGON -170.91503954 62.17657156 -170.9868328...
5      POLYGON -170.86729349 62.198794 -170.93913998 ...
6      POLYGON -170.82080280000002 62.22350131 -170.8...
7      POLYGON -170.77180089 62.24326607 -170.8437531...
8      POLYGON -170.86062438 62.15118278 -170.9323576...
9      POLYGON -170.81283663 62.17340499 -170.8846225...
10     POLYGON -170.76513210999997 62.19565498 -170.8...
11     POLYGON -170.71734430000004 62.21787733 -170.7...
12     POLYGON -170.80844501 62.113211 -170.84993097 ...
13     POLYGON -170.75842137 62.14804389 -170.8301471...
14     POLYGON -170.71071697000002 62.1702661 -170.78...
15     POLYGON -170.66241273000003 62.19083358 -170.7...
16     POLYGON -170.77846933 62.29087706 -170.8505352...
17     POLYGON -170.72571441000

In [20]:
def polysplit(region='polygon(150.099223,2.391097,150.086084,2.422515,150.050573,2.407277,150.064586,2.376241)', get_shapely=False):
    
    spl = region[region.find('(')+1:region.find(')')].split(',')
    px = spl[0::2]
    py = spl[1::2]
    px.append(px[0])
    py.append(py[0])
    px, py = np.cast[float](px), np.cast[float](py)
    
    if get_shapely:
        from shapely.geometry import Polygon
        list = []
        for i in range(len(px)):
            list.append((px[i], py[i]))
        
        poly = Polygon(tuple(list))
        return poly
    else:
        return px, py

In [21]:
n=1
reg_type =  s_regions_DecRA[n].split()[0].lower()
reg_split =  s_regions_DecRA[n].split()
if reg_split[1].startswith('G'):
    coords = '('+','.join(reg_split[2:])+')'
else:
    coords = '('+','.join(reg_split[2:])+')'

test_reg = reg_type+coords 
poly_test1 = polysplit(test_reg, get_shapely=True)

IndexError: index 6 is out of bounds for axis 0 with size 6

In [ ]:
poly_test1

In [ ]:
n=2
reg_type =  s_regions_DecRA[n].split()[0].lower()
reg_split =  s_regions_DecRA[n].split()
if reg_split[1].startswith('G'):
    coords = '('+','.join(reg_split[2:])+')'
else:
    coords = '('+','.join(reg_split[2:])+')'

test_reg2 = reg_type+coords 
poly_test2 = polysplit(test_reg2, get_shapely=True)

In [ ]:
print(poly_test2)

In [ ]:
print(poly_test1.intersection(poly_test2))